In [1]:
import os, json
import random
import pandas as pd # for data stuff
import numpy as np # for importing data i think, not sure anymore
import matplotlib.pyplot as plt #for plotting
from mlxtend.frequent_patterns import apriori #for frequent itemset
from mlxtend.preprocessing import TransactionEncoder #for frequent itemset


# the modules below are used in "Analys Models" section
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from pandas.io.json import json_normalize
from IPython.display import Video

# Analysis of Brawl Stars Data Set

## What is Brawl Stars?

Brawl Stars is a mobile online multiplayer battle game. A player choses a Brawler as game character and plays against other players or AI opponents. Currently there exist 36 different Brawlers, which a player can choose from. Each Brawler has a power level from 1 to 10. With the help of the in-game loot boxes those power levels can be upgraded.
There are several playable modes, which are updated in regular intervals (every 24hours). Of course, it should also be mentioned that each mode has its preferred characters.
There exist several game modes, where you either play solo or in a team of 2 or 3 against other players/teams or AI opponents.

Every groupmember is an active Brawl Stars gamer. That's the reason why we have decided to analyse some interesting insights of the Brawl Stars dataset. In the video below, you can see a small snipet of the mode "Brawlball". Brawlball is a 3 vs 3 match configuration, where your teams needs to score 2 goals against the opponent to win. In case you get attacked and killed by the opponents, you are out of the game for 10 seconds. This increases the chance of the opponent team for scoring an opposing goal. 

In [ ]:
# a short video showing the game mode "Brawl Ball"; 
Video("Brawlball.mp4")

## Analysis Questions

In each game, the main goal of every player is winning. Hence, we focused our question on how to choose your own brawler,
respectively the team composition, to win games. Hence, we eventually came up with the following questions:

1. __Are there some brawlers, which are more often picked or are more likely to win?__
2. __Are there team combinations which are more likely to win?__
3. __Is it possible to predict the star player from the winning team?__ The star player is the player which has performed best,
which is calculated by his number of kills, his total damage and the game-goal performance. The star player gains more points than
the other players.
4. __Is it possible to predict the winning team of a match?__

# Preliminaries

- To process and clean the Brawl Stars Data Set we make use of python libraries which are designed for data processing.
- To collect the Brawl Stars Data Set a Brawl Stars API wrapper called brawlstats is used.

Install brawlstats, a python wrapper to retrieve data from the Brawl Stars API
 - https://brawlstats.readthedocs.io/en/latest/index.html

In [ ]:
!pip install brawlstats

# Data Collection

- We collect battlelogs from the Brawl Stars API.

Import brawlstats 

In [ ]:
import brawlstats
import time

Establish connection to Brawl Stars API. A token resp. API Key is needed. Follow the steps below to obtain a token.
- create an account at: https://developer.brawlstars.com/#/
- get your IPv4 with: https://www.whatismyip.org/
- create an API Key on https://developer.brawlstars.com/#/ for your account and your API Key
- save your API key in this directory in a file named: token_brawlstats.txt
- run the cell below to establish the connection to the API

In [ ]:
path = os.getcwd()
token_file =  open(os.path.join(path, "token_brawlstats.txt"), 'r')
token = token_file.read()
token_file.close()

client = brawlstats.Client(token)

- retrieve the top 200 players
- each player object contains a list of battle logs. Those battle logs contain meta data about the player's most recent played games. Those battlelogs form the raw data used for this project
- If we collect data from the API, the most recent battle logs will be retrieved. Therefore, we decided to store the retrieved data in "battlelog.json" such that we can continue with the same data every time this jupyter notebook is used. However, one could delete the current "battlelog.json" file in order to create a new data set from recently played games
- If you run the cell below a new data set is collected if no curent "battlelog.json" exists. However, if a new data set is collected, some adapations in this projects need to be made.

we assume the skill level of the top 200 players in the game is about the same. Therefore all retrieved battle logs are from players with a very high skill level, making the raw data more consistent and not depending on the skill level of a player


In [ ]:
if not os.path.isfile("battlelogs.json"):
    
    # retrieve the player objects of the top 200 players
    tags_top_200_players = [player["tag"] for player in client.get_rankings(ranking='players', limit=200)]
    top_200_players = [client.get_player(tag) for tag in tags_top_200_players]
    
    # retrive their battlelogs with the corresponding player tag and stor it in "battlelogs.json" file
    with open("battlelogs.json", "w") as battlelog_file:
        for player in top_200_players:
            player_battlelogs = client.get_battle_logs(player.tag)
            for battlelog in player_battlelogs:
                battlelogs[index] = {}
                battlelogs[index]["player_tag"] = player.tag
                battlelogs[index]["battlelog"] = battlelog
                index += 1
        json.dump(battlelogs, battlelog_file)
        

# Data Cleaning

- some of the top players play against each other. Duplicates can be filtered since a battle log contains a unique key = "battle_time"

In [ ]:
# read in the battlelogs.json file
with open("battlelogs.json",'r') as in_file:
    battlelogs = json.load(in_file)
    in_file.close()
    
# initialize list for unique battlelogs
unique_battlelogs = []
used_battle_times = []

# iterate over all battle logs
for key in battlelogs.keys():
    
    # get the battle time of the current battle log which is a unique key
    current_battle_time = battlelogs[key]['battlelog']["battle_time"]
    
    # append the current battle log if 
    # it is not already in the list of unique battle logs
    if not current_battle_time in used_battle_times:
        used_battle_times.append(current_battle_time)
        unique_battlelogs.append(battlelogs[key])

print("The total number of unique battle logs is: {}\n".format(len(unique_battlelogs)))
print("Below is a full battle log: ")    
print(json.dumps(unique_battlelogs[1], sort_keys=True, indent=4))
 
      

# Data Processing
- The Brawlers of the top players are analysed to see if they have most of their Brawlers on the max power level.

In [ ]:
# retrieve the player objects of the top 200 players
tags_top_200_players = [player["tag"] for player in client.get_rankings(ranking='players', limit=200)]
top_200_players = [client.get_player(tag) for tag in tags_top_200_players]


# initialize brawler levels count to 0
# brawlers can be from level 1 to 10
brawler_levels = {}
for i in range(1, 11):
    brawler_levels[i] = 0
    
# loop over the top 200 players
for player in top_200_players:
    
    # loop over the brawlers of the current player
    for brawler in player.brawlers:
        # increment
        brawler_levels[brawler["power"]] += 1

# create a bar plot of the distribution of the brawler power levels
objects = brawler_levels.keys()
y_pos = np.arange(len(objects))
performance = brawler_levels.values()

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.xlabel("Power Level of Brawler")
plt.ylabel('Number of Brawlers')
plt.title('Brawler Levels of top 200 players')

plt.show()

The plot clearly shows that the top 200 players have their brawler mostly on the max power level

# Displaying information about Modes and Maps (Occurence)

 We do want to have the number of games played for the different modes in order to be able to decide which mode to take.
 
 Additionally, we did the same with the different maps, although due to the small number for each map we rather focused on the game mode and decided to further analyse the game mode called "BrawlBall"

In [ ]:

### Retrieves information about the modes played and returns a dictionary with key:value (mode: counter)
modedict = {}
current_mode = ''
for log in unique_battlelogs:
    current_mode = log["battlelog"]["event"]["mode"]
    if current_mode not in modedict.keys():
        modedict[current_mode] = 0
    modedict[current_mode] = modedict[current_mode] + 1
    

### Get the mode with highest games completed in our data
max_key = max(modedict, key=modedict.get)  
print(max_key)
print(modedict)
summode = 0
for val in modedict.values():
    summode = summode + val
print(summode)
    

### Retrieves counter information about the different maps played and returns a dictionary with key:value (map: counter) pairs
mapdict = {}
current_map = ''
for log in unique_battlelogs:
    current_map = log["battlelog"]["event"]["map"]
    if current_map not in mapdict.keys():
        mapdict[current_map] = 0
    mapdict[current_map] = mapdict[current_map] + 1
    
print(mapdict)
summap = 0
for values in mapdict.values():
    summap = summap + values
print(summap)
            

## Creating a Dataframe for further analysis
Since we will be able to perform certain actions on the dataset and want to get rid of the tedious JSON format, we're pulling out the important data for future analysis from the JSON and add it to a Dataframe. Additionally, the Game result (victory, draw, loss) is now numerically encoded.
As a last step, we removed games where a brawler was present at least twice in the same team. As this normally does not occur on games, we wanted to remove data that showed this property.

In [ ]:
### Narrowing down the data to have only variables of interest

brawl_dicts = []
for log in unique_battlelogs:
    if log["battlelog"]["event"]["mode"] == max_key:
        brawl_dicts.append(log)
        
print(len(brawl_dicts))
###print(brawl_dicts)

##removing any unnecessary information from the source

df = pd.DataFrame(columns=['WinningTeam','BattleTime','playerTag','mode','map','result','StarPlayerTag','StarPlayerBrawler','Brawler1Name','Brawler1Tag','Brawler2Name','Brawler2Tag','Brawler3Name','Brawler3Tag','Brawler4Name',
                           'Brawler4Tag','Brawler5Name','Brawler5Tag','Brawler6Name','Brawler6Tag'])

    
for i in range(len(brawl_dicts)):
    log = brawl_dicts[i]
    row = []
    
    if log["battlelog"]["battle"]["result"] == "draw":
        row.append(0)
    elif (log["player_tag"] == log["battlelog"]["battle"]["teams"][0][0]["tag"] or log["player_tag"] == log["battlelog"]["battle"]["teams"][0][1]["tag"] or log["player_tag"] == log["battlelog"]["battle"]["teams"][0][2]["tag"]) and log["battlelog"]["battle"]["result"] == "victory":
        row.append(1)
    else:
        row.append(2)
    row.append(log["battlelog"]["battle_time"])
    row.append(log["player_tag"])
    row.append(log["battlelog"]["event"]["mode"])
    row.append(log["battlelog"]["event"]["map"])
    row.append(log["battlelog"]["battle"]["result"])
    row.append(log["battlelog"]["battle"]["star_player"]["tag"])
    row.append(log["battlelog"]["battle"]["star_player"]["brawler"]["name"])
    row.append(log["battlelog"]["battle"]["teams"][0][0]["brawler"]["name"])
    row.append(log["battlelog"]["battle"]["teams"][0][0]["tag"])
    row.append(log["battlelog"]["battle"]["teams"][0][1]["brawler"]["name"])
    row.append(log["battlelog"]["battle"]["teams"][0][1]["tag"])
    row.append(log["battlelog"]["battle"]["teams"][0][2]["brawler"]["name"])
    row.append(log["battlelog"]["battle"]["teams"][0][2]["tag"])
    row.append(log["battlelog"]["battle"]["teams"][1][0]["brawler"]["name"])
    row.append(log["battlelog"]["battle"]["teams"][1][0]["tag"])
    row.append(log["battlelog"]["battle"]["teams"][1][1]["brawler"]["name"])
    row.append(log["battlelog"]["battle"]["teams"][1][1]["tag"])
    row.append(log["battlelog"]["battle"]["teams"][1][2]["brawler"]["name"])
    row.append(log["battlelog"]["battle"]["teams"][1][2]["tag"])
    df.loc[i] = row

df = df.replace(to_replace = 'victory', value= 1)
df = df.replace(to_replace = 'defeat', value= -1)
df = df.replace(to_replace = 'draw', value= 0)


df.head(10)


### Dropping data where a brawler is present twice in the same team, as this is normally not permitted in games
duplicateBrawlersIndices = []
for index, row in df.iterrows():
    if (row["Brawler1Name"] == row["Brawler2Name"] or row["Brawler1Name"] == row["Brawler2Name"] or row["Brawler2Name"] == row["Brawler3Name"]
        or row["Brawler4Name"] == row["Brawler5Name"] or row["Brawler4Name"] == row["Brawler6Name"] or row["Brawler5Name"] == row["Brawler6Name"]):
        duplicateBrawlersIndices.append(index)
df = df.drop(duplicateBrawlersIndices)



df.to_csv("rawdata.csv",index=False)


### Preparing Data to check brawler-specific Pickrate, meaning the chance (in %) of each brawler getting picked for BrawlBall mode

In [ ]:
## List of all Brawlernames to simplify certain actions

brawlerlist = ['SHELLY','NITA','COLT','BULL','JESSIE','BROCK','DYNAMIKE','BO','TICK','8-BIT','EMZ','EL PRIMO','BARLEY','POCO','ROSA','DARRYL','PENNY','CARL','JACKY','PIPER','PAM','FRANK','BIBI','BEA','RICO','MORTIS','TARA','GENE','MAX','MR. P','SPROUT','SPIKE','CROW','LEON','SANDY','GALE']
print(len(brawlerlist))

## Creating a new Dataframe for Calucalating the Pickrate of each brawler (through duplicate removal and comparing to teams (not games), may not exceed 100%)
newdf = pd.DataFrame(brawlerlist,columns=['Name'])

OccurencesBrawler =[]
TotalGames = []

for brawler in brawlerlist:
    x = list(df.Brawler1Name).count(brawler)
    y = list(df.Brawler2Name).count(brawler)
    z = list(df.Brawler3Name).count(brawler)
    a = list(df.Brawler4Name).count(brawler)
    b = list(df.Brawler5Name).count(brawler)
    c = list(df.Brawler6Name).count(brawler)
    
    total_sum = a + b + c + x + y + z
    OccurencesBrawler.append(total_sum)
    ### As each team may choose every brawler (although premade teams are not allowed to pick duplicates), we derive the TotalTeam Number from the Total games played.
    TotalGames.append(len(df.index)*2)

    
## Adding New columns for calulcation of the "pick rate" (how often is the respective brawler picked)
## Additionally, to perform row-wise transformations, the Number of Total games is written in each row
newdf['Occurence'] = OccurencesBrawler
newdf['TotalGames'] = TotalGames

newdf['PickRate %'] = newdf.Occurence / newdf.TotalGames * 100
newdf.sort_values(by='PickRate %',ascending = False)

### Preparation and Constructing Tables that show brawler-specific Winrate (in %), show how many times the brawler (individually) wins

In [ ]:
## List of all Brawlernames to simplify certain actions

brawlerlist = ['SHELLY','NITA','COLT','BULL','JESSIE','BROCK','DYNAMIKE','BO','TICK','8-BIT','EMZ','EL PRIMO','BARLEY','POCO','ROSA','DARRYL','PENNY','CARL','JACKY','PIPER','PAM','FRANK','BIBI','BEA','RICO','MORTIS','TARA','GENE','MAX','MR. P','SPROUT','SPIKE','CROW','LEON','SANDY','GALE']


## Creating a new Dataframe for Calucalating the Pickrate of each brawler, keep in mind that every non-premade team may have duplicate champions, therefore the Pickrate could theoretically exceed 100% (up to 300%)
windf = pd.DataFrame(brawlerlist,columns=['Name'])

## List that will be filled with the number each brawler has played in the dataset.
GamesPlayed =[]

for brawler in brawlerlist:
    x = list(df.Brawler1Name).count(brawler)
    y = list(df.Brawler2Name).count(brawler)
    z = list(df.Brawler3Name).count(brawler)
    a = list(df.Brawler4Name).count(brawler)
    b = list(df.Brawler5Name).count(brawler)
    c = list(df.Brawler6Name).count(brawler)
    
    total_sum = a + b + c + x + y + z
    GamesPlayed.append(total_sum)
    
winningOccurences = []

In [ ]:
for brawler in brawlerlist:
    brawlwincount = 0
    for index, row in df.iterrows():
        if row["WinningTeam"] == 1 and (row["Brawler1Name"] == brawler or row["Brawler2Name"] == brawler or row["Brawler3Name"] == brawler):
            brawlwincount += 1
        elif row["WinningTeam"] == 2 and (row["Brawler4Name"] == brawler or row["Brawler5Name"] == brawler or row["Brawler6Name"] == brawler):
            brawlwincount += 1
    winningOccurences.append(brawlwincount)

# Appended Games played to the dataframe to make calculations
windf['Games Played'] = GamesPlayed
windf['Games Won'] = winningOccurences
windf['Winrate %'] = windf['Games Won'] / windf['Games Played'] * 100
windf.sort_values(by='Winrate %', ascending=False)

##Side note: It may occur that the teams playing against each other have the same brawler in one game, this would result in a winrate of 50% (if only games where both teams the same brawler were counted).

# Analysis

## Winning Team Combinations

For our second questions - if certain team combinations exists, which win often - we used frequent item set mining.
In a first step, we analysed the team combinations with the specific brawlers. Afterwards, we did the same, but for the brawler types.
Each Brawler has a certain type (Sniper, Thrower, etc.) which is sometimes advantageous on certain maps.

### Brawler-Team Composition

In this part, the specific brawler-team composition in analysed by frequent itemset mining.

In [ ]:
# extract list of winning teams
dfWinTeam = pd.read_csv("rawdata.csv")
winningTeams = []
counter = 0
for index, row in dfWinTeam.iterrows():
    team = []
    if row["WinningTeam"] == 1 and row["result"] != 0:
        team.append(row["Brawler1Name"])
        team.append(row["Brawler2Name"])
        team.append(row["Brawler3Name"])
    elif row["result"] != 0:
        team.append(row["Brawler4Name"])
        team.append(row["Brawler5Name"])
        team.append(row["Brawler6Name"])

    if len(team) != 0:
        winningTeams.append(team)

# transform table in correct format (one column per brawler)
te = TransactionEncoder()
te_ary = te.fit(winningTeams).transform(winningTeams)
dfFreqSetsWinnerBrawler = pd.DataFrame(te_ary, columns=te.columns_)

freq_item_winner_brawler = apriori(dfFreqSetsWinnerBrawler, min_support=0.01, use_colnames=True)
freq_item_winner_brawler['length'] = freq_item_winner_brawler['itemsets'].apply(lambda x: len(x))

# get most supported teams of 3
freq_item_winner_brawler[ (freq_item_winner_brawler['length'] == 3) ].sort_values(by=["support"], ascending=False)

The table shows that there are some team combinations with a support higher than 1%. The most used winning team consists of
Brock, Bea and Penny with a support of 2.7%.

We did the same analysis with loosing teams, however there are no team combination with a support higher than 1%, hence it
is not included.



## Reading the Dataset

In [2]:
df = pd.read_csv("rawdata.csv")
print(df.shape)

(1146, 20)


## Fix most used Map
We want to analyze the different combinations of brawler types. As the combinations depend on the map, we fix our data frame to th most used map.
- first check which map is used the most

In [3]:
pd.value_counts(df['map'])

Backyard Bowl     903
Post Haste        119
Warped Arena       25
Triple Dribble     22
Super Stadium      17
Beach Ball         16
Field Goal         11
Pinhole Punt        9
Well Cut            7
Pinball Dreams      6
Galaxy Arena        4
Penalty Kick        4
Center Field        2
Shoulder Bash       1
Name: map, dtype: int64

Copy initial data frame and remove all games that are played on other maps

In [4]:
df_fixed_map = df.copy()
df_fixed_map = df_fixed_map[df_fixed_map['map'] == 'Backyard Bowl'] 

## Define Brawler Type Lists
We need to define the type for each brawler. The information is obtained from the game itself.

In [5]:

sniper_brawlers = ['COLT','RICO','8-BIT','PIPER','BROCK','BEA','PENNY','MR. P','SPIKE']
thrower_brawlers = ['DYNAMIKE','TICK','BARLEY']
heavyweight_brawlers = ['DARRYL','EL PRIMO','ROSA','BULL','FRANK','JACKY','BIBI']
fighter_brawlers = ['CARL','EMZ','BO','TARA','JESSIE','NITA','SHELLY']
support_brawlers = ['POCO','PAM','GENE','MAX','SPROUT','SANDY','GALE',]
assassin_brawlers = ['MORTIS','CROW','LEON']


## Add Brawler Types to the DataFrame
###### Function to Assign Brawler Type

In [6]:
def assign_brawler_type(brawler):
    if brawler in sniper_brawlers:
        return 'SNIPER'
    elif brawler in thrower_brawlers:
        return 'THROWER'
    elif brawler in heavyweight_brawlers:
        return 'HEAVYWEIGHT'
    elif brawler in fighter_brawlers:
        return 'FIGHTER'
    elif brawler in support_brawlers:
        return 'SUPPORTER'
    elif brawler in assassin_brawlers:
        return 'ASSASSIN'
    else:
        return ''
    

##### Add Brawler Types to DataFrame

In [7]:
df_fixed_map['Brawler1Type'] = df_fixed_map['Brawler1Name'].apply(assign_brawler_type)
df_fixed_map['Brawler2Type'] = df_fixed_map['Brawler2Name'].apply(assign_brawler_type)
df_fixed_map['Brawler3Type'] = df_fixed_map['Brawler3Name'].apply(assign_brawler_type)
df_fixed_map['Brawler4Type'] = df_fixed_map['Brawler4Name'].apply(assign_brawler_type)
df_fixed_map['Brawler5Type'] = df_fixed_map['Brawler5Name'].apply(assign_brawler_type)
df_fixed_map['Brawler6Type'] = df_fixed_map['Brawler6Name'].apply(assign_brawler_type)


## Check if same Type is used by multiple Players
If used by multiple players we add _<<number>> to the type to be able to compute the frequent itemsets

In [8]:
counter = 0
for index, row in df_fixed_map.iterrows():
    if row['Brawler1Type'] == row['Brawler2Type'] and row['Brawler1Type'] != row['Brawler3Type']:
        df_fixed_map.loc[index, 'Brawler2Type'] = row['Brawler2Type']+'_2'
    elif row['Brawler1Type'] == row['Brawler2Type'] and row['Brawler1Type'] == row['Brawler3Type']:
        df_fixed_map.loc[index,'Brawler2Type'] = row['Brawler2Type']+'_2'
        df_fixed_map.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_3'
    elif row['Brawler1Type'] == row['Brawler3Type'] and row['Brawler1Type'] != row['Brawler2Type'] and row['Brawler2Type'] != row['Brawler3Type']:
        df_fixed_map.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_2'
    elif row['Brawler1Type'] != row['Brawler3Type'] and row['Brawler1Type'] != row['Brawler2Type'] and row['Brawler2Type'] == row['Brawler3Type']:
        df_fixed_map.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_2'
df_fixed_map.head(20)



,WinningTeam,BattleTime,playerTag,mode,map,result,StarPlayerTag,StarPlayerBrawler,Brawler1Name,Brawler1Tag,...,Brawler5Name,Brawler5Tag,Brawler6Name,Brawler6Tag,Brawler1Type,Brawler2Type,Brawler3Type,Brawler4Type,Brawler5Type,Brawler6Type
0,1,20200530T122433.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,...,BROCK,#20C2RRRGY,EMZ,#Y220JJJ9,SNIPER,SNIPER_2,SUPPORTER,SNIPER,SNIPER,FIGHTER
1,1,20200530T121808.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,...,MAX,#22L8CLYQU,PENNY,#28U009JPY,SNIPER,SNIPER_2,SUPPORTER,SUPPORTER,SUPPORTER,SNIPER
2,1,20200530T121207.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,...,BROCK,#9QJP9LP,SPIKE,#98VJCL2PQ,SNIPER,SNIPER_2,SUPPORTER,SUPPORTER,SNIPER,SNIPER
3,1,20200530T120359.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,...,FRANK,#2UCV2R0LV,POCO,#22RURYRV0,SNIPER,SNIPER_2,SUPPORTER,HEAVYWEIGHT,HEAVYWEIGHT,SUPPORTER
4,1,20200530T120028.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,...,BROCK,#98JQVGLP9,JACKY,#8822ULJ9Q,SNIPER,SNIPER_2,SUPPORTER,SNIPER,SNIPER,HEAVYWEIGHT
5,1,20200530T115600.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9UR09Q80Y,GENE,GENE,#9UR09Q80Y,...,PIPER,#298R28UG2,BROCK,#QRVCCPPR,SUPPORTER,SNIPER,SNIPER_2,SUPPORTER,SNIPER,SNIPER
6,2,20200530T115304.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PAM,#20JRL29PY,...,GENE,#9UR09Q80Y,BEA,#9U2QJ0P,SUPPORTER,SNIPER,SNIPER_2,SNIPER,SUPPORTER,SNIPER
7,2,20200530T115056.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#20JLGV0QL,...,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,SNIPER,SUPPORTER,SUPPORTER_2,SNIPER,SNIPER,SUPPORTER
8,2,20200530T114747.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9UR09Q80Y,GENE,BEA,#PJJ9P890,...,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,SNIPER,FIGHTER,SUPPORTER,SNIPER,SNIPER,SUPPORTER
11,2,20200530T143201.000Z,#8P80J8RC,brawlBall,Backyard Bowl,-1,#YCC8VGRU,PAM,MR. P,#8P80J8RC,...,PAM,#YCC8VGRU,PIPER,#9JY82J8U,SNIPER,SNIPER_2,SNIPER_3,SUPPORTER,SUPPORTER,SNIPER


## Find Frequent Combinations of Brawler Types
First we want to find the frequent used combinations of brawler types. This is done by computing the frequent itemsets.

In [9]:
combinations = []
counter = 0
for index, row in df_fixed_map.iterrows():
    team1 = []
    team2 = []
    team1.append(row["Brawler1Type"])
    team1.append(row["Brawler2Type"])
    team1.append(row["Brawler3Type"])
    team2.append(row["Brawler4Type"])
    team2.append(row["Brawler5Type"])
    team2.append(row["Brawler6Type"])

    combinations.append(team1)
    combinations.append(team2)

In [10]:
te = TransactionEncoder()
te_ary = te.fit(combinations).transform(combinations)
df_fixed_mapFreqSetsCombinations = pd.DataFrame(te_ary, columns=te.columns_)
df_fixed_mapFreqSetsCombinations


,ASSASSIN,ASSASSIN_2,ASSASSIN_3,FIGHTER,FIGHTER_2,FIGHTER_3,HEAVYWEIGHT,HEAVYWEIGHT_2,HEAVYWEIGHT_3,SNIPER,SNIPER_2,SNIPER_3,SUPPORTER,SUPPORTER_2,SUPPORTER_3,THROWER
0,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
1,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False
1802,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False
1803,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False
1804,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False


In [84]:
frequent_itemsets_combinations = apriori(df_fixed_mapFreqSetsCombinations, min_support=0.001, use_colnames=True)
frequent_itemsets_combinations['length'] = frequent_itemsets_combinations['itemsets'].apply(lambda x: len(x))

NameError: name 'apriori' is not defined

In [85]:
frequent_itemsets_combinations[ (frequent_itemsets_combinations['length'] == 3) ].sort_values(by=["support"], ascending=False)




NameError: name 'frequent_itemsets_combinations' is not defined

## Find Frequent Winning Combinations of Brawler Types
We want to find the frequent winning combinations of brawler types. This is done by computing the frequent itemsets.

In [60]:
winningCombinations = []
counter = 0
for index, row in df_fixed_map.iterrows():
    team = []
    if row["WinningTeam"] == 1 and row["result"] != 0:
        team.append(row["Brawler1Type"])
        team.append(row["Brawler2Type"])
        team.append(row["Brawler3Type"])
    elif row["result"] != 0:
        team.append(row["Brawler4Type"])
        team.append(row["Brawler5Type"])
        team.append(row["Brawler6Type"])

    if len(team) != 0:
        winningCombinations.append(team)

In [61]:
te = TransactionEncoder()
te_ary = te.fit(winningCombinations).transform(winningCombinations)
df_fixed_mapFreqSetsWinner = pd.DataFrame(te_ary, columns=te.columns_)
df_fixed_mapFreqSetsWinner

NameError: name 'TransactionEncoder' is not defined

In [62]:
frequent_itemsets_winner = apriori(df_fixed_mapFreqSetsWinner, min_support=0.001, use_colnames=True)
frequent_itemsets_winner['length'] = frequent_itemsets_winner['itemsets'].apply(lambda x: len(x))

NameError: name 'apriori' is not defined

In [63]:
frequent_itemsets_winner[ (frequent_itemsets_winner['length'] == 3) ].sort_values(by=["support"], ascending=False)

NameError: name 'frequent_itemsets_winner' is not defined

## Find Frequent Loosing Combinations of Brawler Types
We want to find the frequent loosing combinations of brawler types. This is done by computing the frequent itemsets.

In [64]:
loosingCombinations = []
counter = 0
for index, row in df_fixed_map.iterrows():
    team = []
    if row["WinningTeam"] == 1 and row["result"] != 0:
        team.append(row["Brawler4Type"])
        team.append(row["Brawler5Type"])
        team.append(row["Brawler6Type"])
    elif row["result"] != 0:
        team.append(row["Brawler1Type"])
        team.append(row["Brawler2Type"])
        team.append(row["Brawler3Type"])

    if len(team) != 0:
        loosingCombinations.append(team)

In [65]:
te = TransactionEncoder()
te_ary = te.fit(loosingCombinations).transform(loosingCombinations)
df_fixed_mapFreqSetsLooser = pd.DataFrame(te_ary, columns=te.columns_)
df_fixed_mapFreqSetsLooser

NameError: name 'TransactionEncoder' is not defined

In [66]:
frequent_itemsets_looser = apriori(df_fixed_mapFreqSetsLooser, min_support=0.001, use_colnames=True)
frequent_itemsets_looser['length'] = frequent_itemsets_looser['itemsets'].apply(lambda x: len(x))

NameError: name 'apriori' is not defined

In [67]:
frequent_itemsets_looser[ (frequent_itemsets_looser['length'] == 3) ].sort_values(by=["support"], ascending=False)



NameError: name 'frequent_itemsets_looser' is not defined

## Check Winning Percentage for Combinations
We want to analyze how often a combination wins relatively to the usage of this combination. Therefore we compute the relative winning percentage for every used combination

In [68]:
length = len(df_fixed_map)
winning_percentage = []
for index, row in frequent_itemsets_combinations.iterrows():
    for idx, winner in frequent_itemsets_winner.iterrows():
        new_row = []    
        if row['itemsets'] == winner['itemsets']:
            total_number_of_combination = length * 2 * row['support']
            total_number_of_wins = length * winner['support']
            win_percentage = total_number_of_wins / total_number_of_combination
            new_row.append(winner['itemsets'])
            new_row.append(win_percentage)
            new_row.append(total_number_of_combination)
            winning_percentage.append(new_row)
winning_percentage = pd.DataFrame(winning_percentage).rename(columns={0: "itemsets", 1: 'winning percentage', 2: 'times picked'})
winning_percentage = winning_percentage[winning_percentage['itemsets'].map(len) == 3].sort_values(by=['winning percentage'], ascending=False)
winning_percentage
            

NameError: name 'frequent_itemsets_combinations' is not defined

## Correlation between Winning-Rate and Usage of a Combination
First we want to remove outliers

In [69]:
cleaned_winning_percentage = winning_percentage.copy()
def remove_outlier(df_fixed_map_in, col_name):
    q1 = df_fixed_map_in[col_name].quantile(0.25)
    q3 = df_fixed_map_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    print(fence_high)
    print(fence_low)
    df_fixed_map_out = df_fixed_map_in.loc[(df_fixed_map_in[col_name] > fence_low) & (df_fixed_map_in[col_name] < fence_high)]
    return df_fixed_map_out
remove_outlier(cleaned_winning_percentage, 'times picked')

TypeError: list indices must be integers or slices, not str

As we see no entries are removed

In [70]:
plt.scatter(winning_percentage['times picked'], winning_percentage['winning percentage'])
plt.show()


TypeError: list indices must be integers or slices, not str

Next we want to check whether there is a correlation between usage of a combination and the winning rate.

In [71]:
cleaned_winning_percentage['times picked'].corr(cleaned_winning_percentage['winning percentage'])

TypeError: list indices must be integers or slices, not str

As we see the correlation is slightly positive with 0.208. Thus, there is a small correlation between the usage of a combination of brawler types and the winning rate. This indicates that it is more probable to win with a certain combination of brawler types on specific maps.

## Starplayer of the Winning Team

The goal of our third question is to predict the "StarPlayer" of the Winning Team in the mode "BrawlBall". We had the following approach:

- 1. We need to extract the Brawlers of the winning team, since the Starplayer can only come from the winning team
- 2. The variables "winningTeams", "map" and the "StarPlayer" of the match needs to to be encoded to perform a classification. The idea is that "winningTeam" and "map" defines the target variable "StarPlayer"
- 3. We will try to predict the "StarPlayer" of the match with a RandomForestClassifier and interpret the result

In [ ]:
#read dataset
dfStarPlayer = pd.read_csv("rawdata.csv")

# extract winning teams
winningTeams = []

for index, row in dfStarPlayer.iterrows():
    team = []

    if row["WinningTeam"] == 1:
        team.append(row["Brawler1Name"])
        team.append(row["Brawler2Name"])
        team.append(row["Brawler3Name"])
    else:
        team.append(row["Brawler4Name"])
        team.append(row["Brawler5Name"])
        team.append(row["Brawler6Name"])

    # sort teams that brawlers are in same order
    team = sorted(team)

    # concat teams to one string
    teamString = "_".join(team)
    winningTeams.append(teamString)

# encode string for classifier
dfStarPlayerFinal = pd.DataFrame(columns=["Team", "map"])
dfStarPlayerFinal["Team"] = winningTeams
dfStarPlayerFinal["map"] =dfStarPlayer["map"].copy()
dfStarPlayerFinal["StarPlayer"] = dfStarPlayer["StarPlayerBrawler"].copy()

#encode teams
encoding = LabelEncoder()
encoding.fit(winningTeams)
dfStarPlayerFinal["Team"] = encoding.transform(dfStarPlayerFinal["Team"].values)

#encode brawler
encoding = LabelEncoder()
encoding.fit(dfStarPlayerFinal["StarPlayer"].values)
dfStarPlayerFinal["StarPlayer"] = encoding.transform(dfStarPlayerFinal["StarPlayer"].values)

# encode maps
encoding = LabelEncoder()
encoding.fit(dfStarPlayerFinal["map"].values)
dfStarPlayerFinal["map"] = encoding.transform(dfStarPlayerFinal["map"].values)

# extract features
x_features_starplayer = []
starPlayers = []

for index, row in dfStarPlayerFinal.iterrows():
    features = []
    features.append(row["map"])
    features.append(row["Team"])

    x_features_starplayer.append(features)
    starPlayers.append(row["StarPlayer"])

# create classifier
randomStarPlayerTree = RandomForestClassifier()
randomStarPlayerTree = randomStarPlayerTree.fit(x_features_starplayer, starPlayers)

scores = cross_val_score(randomStarPlayerTree, x_features_starplayer, starPlayers, scoring='accuracy')
acc = np.mean(scores)

print("Accuracy: {0}".format(acc))

The accuracy of our model is around 21% with a RandomForestClassifier, which is a bad performance. A winner Team consists of 3 Brawlers. This means the chance of becoming the StarBrawler is 33%. 

To analyse the cause of the warning above, we need to have a look on the number of Starplayer counts for each character in our dataset:

In [ ]:
print(dfStarPlayerFinal.loc[:,"StarPlayer"].value_counts())

Three Brawlers (class) in y have less than 5 occurences, which is less than n_splits=5 for the RandomForestClassifier. As an optimization problem, one could try to set a minimum occurence (e.g. 10). This could help to get a better performance & accurracy.  

## Win-Prediction Model

In the last analysis, we build a model which should predict the winning team according to the brawlers in each team and
the map. The game mode is still fixed to Brawl Ball. From the previous analysis, we saw that certain team compositions are chosen
more often, hence we assume that the team composition plays a large role for winning (as the data is from the Top 200 players, their game
skill should be roughly equal). Furthermore, we assume that the map also has an influence on who wins the game, as certain brawlers
may have an advantage. Thus, we chose these variables as features.
Lastly, we use the RandomTreeClassifier as a model.

In a first step, we encoded the data into a form that is usable for the model.

In [ ]:
dfWinPrediction = pd.read_csv("rawdata.csv")
firstTeams = []
secondTeams = []

for index, row in dfWinPrediction.iterrows():
    team1 = []
    team2 = []

    team1.append(row["Brawler1Name"])
    team1.append(row["Brawler2Name"])
    team1.append(row["Brawler3Name"])

    team2.append(row["Brawler4Name"])
    team2.append(row["Brawler5Name"])
    team2.append(row["Brawler6Name"])

    # sort teams that brawlers are in same order
    team1 = sorted(team1)
    team2 = sorted(team2)

    # concat teams to one string
    team1string = "_".join(team1)
    team2string = "_".join(team2)

    firstTeams.append(team1string)
    secondTeams.append(team2string)

# encode string for classifier
dfFinal = pd.DataFrame(columns=["Team1", "Team2", "map"])
dfFinal["Team1"] = firstTeams
dfFinal["Team2"] = secondTeams
dfFinal["map"] =dfWinPrediction["map"].copy()
dfFinal["WinningTeam"] = dfWinPrediction["WinningTeam"].copy()

encoding = LabelEncoder()
encoding.fit(firstTeams + secondTeams)
dfFinal["Team1"] = encoding.transform(dfFinal["Team1"].values)
dfFinal["Team2"] = encoding.transform(dfFinal["Team2"].values)

# encode maps
encoding = LabelEncoder()
encoding.fit(dfFinal["map"].values)
dfFinal["map"] = encoding.transform(dfFinal["map"].values)

dfFinal.head(5)

Then, we extracted the features from the dataset for the classifier plus created and trained the RandomForestClassifier.

In [ ]:
x_features_winner = []
winners = []

for index, row in dfFinal.iterrows():
    features = []
    features.append(row["map"])
    features.append(row["Team1"])
    features.append(row["Team2"])

    x_features_winner.append(features)
    winners.append(row["WinningTeam"])

# create classifier
randomWinnerTree = RandomForestClassifier()
randomWinnerTree = randomWinnerTree.fit(x_features_winner, winners)

Lastly, we cross validated the the tree in order to get its accurarcy

In [ ]:
scores = cross_val_score(randomWinnerTree, x_features_winner, winners, scoring='accuracy')
acc = np.mean(scores)

print("Accuracy: {0}".format(acc))

We get an average accuracy of about __60%__ for predicting the winner of a game with this model.

# Discussion or whatever

- write down our insights or maybe do some further coding in the cell below


